# Visualizing GO term drift results in Cytoscape

Tong Shu Li

Here we will prepare the data necessary to visualize the GO term drift results in Cytoscape. The relations between GO terms are hidden when the results are presented as a table. By displaying the results graphically, it will be easier to see the similarities between GO terms and the genes that they contain.

In [1]:
import pandas as pd
import sys

from collections import defaultdict

In [2]:
sys.path.append("../..")

In [3]:
from src.gene_ont import group_genes
from src.gene_ont import load_annotations
from src.gene_ont import parse_go_defn
from src.gene_ont import parse_go_links

from src.util import union

---

## Read GO term definitions

In [4]:
go_defn = parse_go_defn("../../data/gene_ontology/go.obo")

In [5]:
go_defn.shape

(44471, 4)

In [6]:
go_defn.head()

,go_id,go_name,namespace,obsolete
0,GO:0000001,mitochondrion inheritance,biological_process,False
1,GO:0000002,mitochondrial genome maintenance,biological_process,False
2,GO:0000003,reproduction,biological_process,False
3,GO:0000005,obsolete ribosomal chaperone activity,molecular_function,True
4,GO:0000006,high-affinity zinc uptake transmembrane transp...,molecular_function,False


In [7]:
go_defn.query("~obsolete")["go_id"].nunique()

42514

---

## Read GO annotations for C. elegans

In [8]:
floc = "../../data/gene_ontology/gene_association.wb"
go_data = load_annotations(floc)

In [9]:
go_data.head()

,database_id,db_obj_symbol,qualifier,go_id,db_ref,evidence,with_from,aspect,db_obj_name,db_obj_syn,db_obj_type,taxon,date,assigned_by,annot_ext,gene_prod_id
0,WBGene00000001,aap-1,NaN,GO:0005623,WB_REF:WBPaper00005614|PMID:12393910,IDA,NaN,C,NaN,Y110A7A.10,gene,taxon:6239,20060302,WB,NaN,NaN
1,WBGene00000001,aap-1,NaN,GO:0005942,GO_REF:0000033,IBA,PANTHER:PTN000806614,C,NaN,Y110A7A.10,gene,taxon:6239,20150227,GO_Central,NaN,NaN
2,WBGene00000001,aap-1,NaN,GO:0005942,GO_REF:0000002,IEA,InterPro:IPR001720,C,NaN,Y110A7A.10,gene,taxon:6239,20150826,WB,NaN,NaN
3,WBGene00000001,aap-1,NaN,GO:0008286,WB_REF:WBPaper00005614|PMID:12393910,IMP,NaN,P,NaN,Y110A7A.10,gene,taxon:6239,20060302,WB,NaN,NaN
4,WBGene00000001,aap-1,NaN,GO:0008340,WB_REF:WBPaper00005614|PMID:12393910,IMP,NaN,P,NaN,Y110A7A.10,gene,taxon:6239,20060302,WB,NaN,NaN


## Propagate annotations up GO hierarchy

In [10]:
direct_annots = defaultdict(
    set,
    {
        go_term: set(df["database_id"])
        for go_term, df in go_data.groupby("go_id")
    }
)

children = parse_go_links("../../data/gene_ontology/go.obo")
annots = group_genes(children, direct_annots, ["is_a", "part_of"])

In [11]:
len(annots)

42514

In [12]:
# number of non empty go terms
sum(not not genes for go_id, genes in annots.items())

8042

There are 8042 GO terms with at least one gene annotation.

## Read sequenced genes

In [13]:
sequenced = pd.read_csv("GO_annotations.tsv", sep = '\t')

## Read results

In [14]:
info = pd.read_csv("../../results/PMID26623667/GO_drift_results.tsv", sep = '\t')

In [15]:
info.head()

,go_id,go_name,num_genes,num_seq,adj_rsq,rsq_pile,rsq_nlpval,frac_seq,log_day_error,log_day_error_percent,log_day_est,log_day_est_pile,log_day_pval,mianserin_error,mianserin_error_percent,mianserin_est,mianserin_est_pile,mianserin_pval,score
0,GO:0000018,regulation of DNA recombination,10,10,0.546393,37.31,0.202802,1.000000,0.019023,23.883692,0.079648,4.23,6.188005e-04,0.029473,26.616939,-0.110729,84.29,0.001571,0.295796
1,GO:0000027,ribosomal large subunit assembly,18,18,0.514729,10.66,0.048954,1.000000,0.018263,24.572996,0.074321,0.41,7.976415e-04,0.028295,29.491808,-0.095942,93.25,0.003477,0.081089
2,GO:0000028,ribosomal small subunit assembly,13,13,0.473093,26.75,0.135192,1.000000,0.006575,27.078080,0.024283,0.57,1.804968e-03,0.010187,30.747066,-0.033133,93.04,0.004688,0.169005
3,GO:0000041,transition metal ion transport,30,22,0.819520,70.35,0.527975,0.733333,0.045938,10.722788,0.428414,75.57,4.263056e-08,0.071173,27.349412,-0.260235,63.45,0.001954,1.337620
4,GO:0000045,autophagosome assembly,23,22,0.277683,1.96,0.008597,0.956522,0.023087,36.927176,0.062519,0.18,1.492262e-02,0.035769,47.776697,-0.074867,94.74,0.051648,0.032846


## Distance metric

We want to know how similar two GO terms are based on the genes that are annotated with that GO term. For simplicity, we will use the well established Jaccard metric.

In [16]:
def jaccard(term_A, term_B):
    """Compute the Jaccard index of two GO terms."""
    return len(annots[term_A] & annots[term_B]) / len(annots[term_A] | annots[term_B])

---

## Create edges for Cytoscape

For the visualization we will create, we will have the following representation:

Nodes:
1. Nodes represent individual GO terms
2. Node size represents the number of gene annotations
3. Node transparency is inversely correlated with drift model score (darker nodes score higher)

Edges:
1. Edge darkness represents Jaccard value of genes between the two nodes
2. Edge type represents GO relationship type (is_a, part_of)
3. Edge direction reflects the direction in the Gene Ontology

To limit the complexity of the resulting visualization, we will limit the number of nodes as follows:
1. Nodes must have 10 to 500 sequenced genes (original GO term criteria)
2. Nodes must be in the top 10% of scoring nodes
3. Nodes must be within two edges of a top scoring node

### Generate all edges

In [17]:
tested = set(info["go_id"])

edges = []
for parent in tested:
    kids = children[parent]
    
    for edge in ['is_a', 'part_of']:
        for child in kids[edge]:
            if child in tested:
                weight = jaccard(parent, child)
                
                key = "{}_{}_{}".format(child, edge, parent)                
                
                edges.append((key, child, edge, parent, weight))
                
cols = ['edge_key', 'source_node', 'edge_type', 'target_node', 'geneset_similarity']
edges = pd.DataFrame(edges, columns = cols)

In [18]:
edges.shape

(2612, 5)

In [19]:
edges.head()

,edge_key,source_node,edge_type,target_node,geneset_similarity
0,GO:0018107_is_a_GO:0006468,GO:0018107,is_a,GO:0006468,0.027897
1,GO:0023014_is_a_GO:0006468,GO:0023014,is_a,GO:0006468,0.122318
2,GO:0018108_is_a_GO:0006468,GO:0018108,is_a,GO:0006468,0.197425
3,GO:0046777_is_a_GO:0006468,GO:0046777,is_a,GO:0006468,0.133047
4,GO:0018105_is_a_GO:0006468,GO:0018105,is_a,GO:0006468,0.276824


In [20]:
edges["edge_type"].value_counts()

is_a       2287
part_of     325
Name: edge_type, dtype: int64

### Filter edges

In [21]:
# get the top N percent of GO terms by score

fraction = 0.15

loc = int(fraction * len(info))
top = info.sort_values("score", ascending = False)

top_nodes = set(top.iloc[:loc]["go_id"])

In [22]:
len(top_nodes)

267

In [23]:
def dfs(cur_node, dist):
    """Return a set of nodes within two edges of the starting node."""
    
    if dist >= 1:
        return children[cur_node]["is_a"] | children[cur_node]["part_of"]
    
    return union(
        dfs(child, dist+1)
        for edge_type in ["is_a", "part_of"]
            for child in children[cur_node][edge_type]
    )

In [24]:
neighbours = union(dfs(node, 0) for node in top_nodes)

In [25]:
len(neighbours)

4262

In [26]:
all_nodes = (neighbours | top_nodes) & tested

In [27]:
len(all_nodes)

329

### Filter edges and write to file

In [28]:
sub_edges = edges.query("source_node in @all_nodes & target_node in @all_nodes")

In [29]:
sub_edges.shape

(221, 5)

In [30]:
sub_edges.to_csv("../../results/PMID26623667/cytoscape/edges.tsv", sep = '\t', index = False)

---

## Create information about nodes for Cytoscape

In [31]:
nodes = info[["go_id", "go_name", "num_seq", "rsq_pile", "score"]]

sub_nodes = nodes.query("go_id in @all_nodes")
sub_nodes.to_csv("../../results/PMID26623667/cytoscape/nodes.tsv", sep = '\t', index = False)